<a href="https://colab.research.google.com/github/kirang77/Codsoft/blob/main/recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas numpy scikit-learn


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import zipfile
import io
import requests

url = 'http://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
r = requests.get(url)
r.raise_for_status()

with zipfile.ZipFile(io.BytesIO(r.content)) as z:
    for filename in z.namelist():
        if filename.endswith('ratings.csv'):
            with z.open(filename) as f:
                df = pd.read_csv(f, usecols=['userId', 'movieId', 'rating'])
            break

user_movie_matrix = df.pivot(index='userId', columns='movieId', values='rating').fillna(0)

user_similarity = cosine_similarity(user_movie_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)

def get_recommendations(user_id, user_movie_matrix, user_similarity_df, num_recommendations=5):
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).index[1:]
    recommended_movies = user_movie_matrix.loc[similar_users].mean().sort_values(ascending=False).index[:num_recommendations]
    return recommended_movies

train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

train_user_movie_matrix = train_data.pivot(index='userId', columns='movieId', values='rating').fillna(0)

train_user_similarity = cosine_similarity(train_user_movie_matrix)
train_user_similarity_df = pd.DataFrame(train_user_similarity, index=train_user_movie_matrix.index, columns=train_user_movie_matrix.index)

recommended_movies_train = get_recommendations(user_id=1, user_movie_matrix=train_user_movie_matrix, user_similarity_df=train_user_similarity_df)

url_movies = 'http://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
r_movies = requests.get(url_movies)
r_movies.raise_for_status()

with zipfile.ZipFile(io.BytesIO(r_movies.content)) as z:
    for filename in z.namelist():
        if filename.endswith('movies.csv'):
            with z.open(filename) as f:
                movies = pd.read_csv(f, usecols=['movieId', 'title'])
            break

recommended_movie_titles = movies[movies['movieId'].isin(recommended_movies_train)]
print("Recommended Movies Titles (Train Data):")
print(recommended_movie_titles)


Recommended Movies Titles (Train Data):
      movieId                             title
257       296               Pulp Fiction (1994)
277       318  Shawshank Redemption, The (1994)
314       356               Forrest Gump (1994)
510       593  Silence of the Lambs, The (1991)
1939     2571                Matrix, The (1999)
